# IRIS PROCESSING NOTEBOOK for Arctic Template

## ReadMe : 
- This notebook is designed for the spreadsheet of the Arctic Template.
- A few guidelines about the snow pit spreadsheet :
    - IRIS version has to be specified as IRIS# in the IRIS sheet of the pit spreadsheet
    - NoData is filled as 9999 in the spreadsheet when logging the .txt file IRIS provides

<br/><br/>

Author : Paul Billecocq, GRIMP, Université de Sherbooke QC Canada
modif Julien Meloche

In [ ]:
import iris_core as iris
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook

### 1. Input spreadsheet path

In [ ]:
FILENAME = 'D:CB-2022-radiometer/snowpit_SSAtoProcess/SBRSP39_20220423_1215.xlsx' #fill in with your specific filename (it has to be placed in the same folder as the notebook)

#specify iris version from sheet
iris_sheet = "IRIS2"
iris_version = iris_sheet[:-1] + '_' + iris_sheet[-1:]
# Read IRIS data from the IRIS sheet of the file
data = pd.read_excel(FILENAME, sheet_name=iris_sheet)

### 2. Processing

#### IRIS data retrieval, cleaning no data, and average reflectance voltages

In [ ]:
# List cleaning function
def nanify_list(input_list):
    return [np.nan if x == 9999 else x for x in input_list]


# Calibration data retrieval
spectralon = data["Spectralon (%)"].dropna().tolist()
spectralon = np.array(spectralon,dtype=float)
calibration_one = nanify_list(data["Calibration measured IRIS (mV)"].dropna().tolist())
calibration_two = nanify_list(data["Other calibration measured IRIS (mV)"].dropna().tolist())
if np.isnan(np.mean(calibration_one))== True:
    calibration_one = nanify_list(data["Calibration multimeter (mV)"].dropna().tolist())
    calibration_two = nanify_list(data["Other calibration multimeter (mV)"].dropna().tolist())

calibration_list = [calibration_one, calibration_two]
calibration_scans = np.array([calib for calib in calibration_list if len(calib) > 0 ])

# Calibration voltage averaging
calibration = np.nanmean(calibration_scans, axis=0)
calibration = np.array(calibration,dtype=float)

# Measures data retrieval
height = data["Height (cm)"].dropna().tolist()
measures = nanify_list(data['Measured IRIS (mV)'].dropna().tolist())
if np.isnan(np.mean(measures)) == True:
    measures = nanify_list(data["Multimeter (mV)"].dropna().tolist())
measures = np.array(measures)

In [ ]:
measures

#### Compute IRIS parameters

In [ ]:
# Voltage to reflectance polynomial fit
polynom = iris.calibration_polynom_fit(spectralon, calibration)

In [ ]:
# Compute reflectance
reflectance = iris.voltage_to_reflectance(measures, polynom)

In [ ]:
# Compute SSA from reflectance
ssa = iris.reflectance_to_ssa(reflectance, iris_version)

In [ ]:
# Compute optical radius from SSA
optical_radius = iris.ssa_to_optical_radius(ssa)

#### Processed data preview

In [ ]:
measures_df = pd.DataFrame.from_dict({
    'height': height,
    'voltage': measures,
    'reflectance': reflectance,
    'ssa': ssa,
    'optical radius': optical_radius
})

measures_df

### 3. Write processed data in the spreadsheet

In [ ]:
# Spreadsheet initialization
workbook = load_workbook(FILENAME) 
iris_sheet_obj = workbook[iris_sheet]

for i in range(len(measures)):
    iris_sheet_obj.cell(row = i+2, column = 9).value = reflectance[i]
    iris_sheet_obj.cell(row = i+2, column = 10).value = ssa[i]
    iris_sheet_obj.cell(row = i+2, column = 11).value = optical_radius[i]

# Write staged changes in the spreadsheet
workbook.save(FILENAME.split('.')[0] + '_IRIS_processed.xlsx')

### Batch process folder

In [ ]:
#function to batch process a folder
# use with next cell
def batchProcess_SSA(FILENAME):
    
    #specify iris version
    iris_sheet = "IRIS2"
    iris_version = iris_sheet[:-1] + '_' + iris_sheet[-1:]
    # Read IRIS data from the IRIS sheet of the file
    data = pd.read_excel(FILENAME, sheet_name=iris_sheet)
    # List cleaning function
    def nanify_list(input_list):
        return [np.nan if x == 9999 else x for x in input_list]


    # Calibration data retrieval
    spectralon = data["Spectralon (%)"].dropna().tolist()
    spectralon = np.array(spectralon,dtype=float)
    calibration_one = nanify_list(data["Calibration measured IRIS (mV)"].dropna().tolist())
    calibration_two = nanify_list(data["Other calibration measured IRIS (mV)"].dropna().tolist())
    if np.isnan(np.mean(calibration_one))== True:
        calibration_one = nanify_list(data["Calibration multimeter (mV)"].dropna().tolist())
        calibration_two = nanify_list(data["Other calibration multimeter (mV)"].dropna().tolist())

    calibration_list = [calibration_one, calibration_two]
    calibration_scans = np.array([calib for calib in calibration_list if len(calib) > 0 ])

    # Calibration voltage averaging
    calibration = np.nanmean(calibration_scans, axis=0)
    calibration = np.array(calibration,dtype=float)

    # Measures data retrieval
    height = data["Height (cm)"].dropna().tolist()
    measures = nanify_list(data['Measured IRIS (mV)'].dropna().tolist())
    if np.isnan(np.mean(measures)) == True:
        measures = nanify_list(data["Multimeter (mV)"].dropna().tolist())
    measures = np.array(measures)
    polynom = iris.calibration_polynom_fit(spectralon, calibration)
    reflectance = iris.voltage_to_reflectance(measures, polynom)
    ssa = iris.reflectance_to_ssa(reflectance, iris_version)
    optical_radius = iris.ssa_to_optical_radius(ssa)
    measures_df = pd.DataFrame.from_dict({
    'height': height,
    'voltage': measures,
    'reflectance': reflectance,
    'ssa': ssa,
    'optical radius': optical_radius
    })
    # Spreadsheet initialization
    workbook = load_workbook(FILENAME) 
    iris_sheet_obj = workbook[iris_sheet]

    for i in range(len(measures)):
        iris_sheet_obj.cell(row = i+2, column = 9).value = reflectance[i]
        iris_sheet_obj.cell(row = i+2, column = 10).value = ssa[i]
        iris_sheet_obj.cell(row = i+2, column = 11).value = optical_radius[i]

    # Write staged changes in the spreadsheet
    workbook.save(FILENAME.split('.')[0] + '_IRIS_processed.xlsx')

In [ ]:
import os
#specify data folder for batch process
data_folder = "D:CB-2022-radiometer/snowpit_SSAtoProcess"

for file in os.listdir(data_folder):
    if ".xlsx" in file:
        try:
            batchProcess_SSA(os.path.join(data_folder,file))
        except:
            print(f'error with {file}')